VIII.  Spherical Harmonic Spectra
===================

**Summary:**    Spherical harmonic spectra sampled at discrete radii, with the complex output quantities $q_{\ell,ij}^m$ are defined such that 

$q_{\ell,ij}^m = \int_0^{2\pi}\int_0^\pi f(r_j,\theta,\phi) Y_\ell^m(\theta,\phi)\,\mathrm{sin}\theta\, d\theta\, d\phi$

for each requested output $f_i$ and each requested output radius $r_j$.  Here, $Y_\ell^m(\theta,\phi)$ is the spherical harmonic function of degree $\ell$ and order $m$.

**Subdirectory:**  Shell_Spectra

**main_input prefix:** shellspectra

**Python Classes:** 

* Shell_Spectra :  Complete data structure associated with Shell_Spectra outputs.
* PowerSpectrum :  Reduced data structure -- contains power spectrum of velocity and/or magnetic fields only.

**Additional Namelist Variables:**  

* shellspectra_levels (indicial) : indices along radial grid at which to output spectra.

* shellspectra_levels_nrm (normalized) : normalized radial grid coordinates at which to output spectra.


The shell-spectra output type allows us to examine the spherical harmonic decomposition of output variables at discrete radii.

Examining the *main_input* file, we see that the following output values have been denoted for the Shell Spectra (see *rayleigh_output_variables.pdf* for mathematical formulae):


| Menu Code  | Description |
|------------|-------------|
| 1          | Radial Velocity |
| 2          | Theta Velocity |
| 3          | Phi Velocity  |



Spherical harmonic spectra can be read into Python using either the **Shell_Spectra** or **PowerSpectrum** classes.  

The **Shell_Spectra** class provides the full complex spectra, as a function of degree ell and azimuthal order m, for each specified output variable.   It possesses an attribute named *lpower* that contains the associated power for each variable, along with its m=0 contributions separated and removed.

**Note:** Because the $f_i$'s represent real (not complex) data, the spectrum for negative $m$-values is just the complex conjugate of that for the positive $m$-values.  For this reason, on the positive $m$-values are stored to save space.  For $m>0$, however, the amplitude of the spectrum is increased by a factor of $\sqrt{2}$ so that the total *power* at a given $\ell$ is preserved. 

We illustrate how to use this class below.  As usual, we call the help() function to display the docstrings that describe the different data structures embodied by each class.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

model_type = 2 # 1 for anelastic example, 2 Boussinesq example
base_dir = '/home/nfeatherstone/runs/cig_vis/'
font_size=14     # Font size for plot labels 

if (model_type == 1):
    model_dir = base_dir+'anelastic/'
    # Define some units for plotting purposes
    eunits = r'(erg cm$^{-3}$)'                # energy density
    tunits = '(s)'                             # time
    vunits = r'(cm s$^{-1}$)'                  # velocity
    dunits = '(cm)'                            # distance
    thermal_label = r' Specific Entropy '      # specific entropy
    thermal_units = r'(erg g$^{-1}$ K$^{-1}$)' # specific entropy
    lunits = r'(erg s$^{-1}$)'                 # energy / time (luminosity)
    funits = '(nHz)'                           # Frequency units
    mfunits = '(g cm$^{-2}$ s$^{-1}$)'
    
    # Next, we set some timestep ranges for plotting purposes
    imin = 0             # minimum iteration number to process for time series
    imax = 10000000      # maximum iteration number to process for time series
    
if (model_type == 2):
    model_dir = base_dir+'Boussinesq/'
    # Define some units for plotting purposes
    eunits = ''                          # energy density
    tunits = '(viscous diffusion times)' # time
    vunits = ''                          # velocity
    dunits = ''                          # distance
    thermal_label = ' Temperature ' 
    thermal_units = '' 
    lunits = ''                          # energy / time (luminosity)
    funits = ''                          # Frequency units
    mfunits = ''
    
    # Next, we set some timestep ranges for plotting purposes
    imin = 0             # minimum iteration number to process for time series
    imax = 10000000      # maximum iteration number to process for time series

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import ticker
import numpy
from rayleigh_diagnostics import Shell_Spectra, build_file_list

files = build_file_list(imin,imax,path=model_dir+'Shell_Spectra')
nf = len(files)
ss = Shell_Spectra(files[nf-1],path='')
help(ss)

First, let's illustrate how to plot the velocity power spectrum at a single radius and time.  Specifically, we want to plot

$P_\ell(r,t) = \sum\limits_{j=r,\theta,\phi}{}\sum\limits_{m=-\ell}^{\ell} v_{j,\ell}^m\, v_{j,\ell}^{m*}$ 

This is easily done with the *lpower* attribute of the class, where the summation over m in the formula above has already been carried out.  The *lpower* array stores the total power (index 0), that associated with the axisymmetric motions (m = 0, index 1) and their difference (i.e., convective power,  index 2).

In [ ]:
#Construct the velocity power spectrum (integrated over ell)
vr = ss.lut[1]
vt = ss.lut[2]
vp = ss.lut[3]

rind = 0 # use the uppermost radius output
tind = 0 # there is only one timestep because we are using time-averaged data

vr_power = ss.lpower[:,rind,vr,tind,:]
vt_power = ss.lpower[:,rind,vt,tind,:]
vp_power = ss.lpower[:,rind,vp,tind,:]

power = vr_power+vt_power+vp_power

fig, ax = plt.subplots(nrows=3, figsize=(10,8))
plt.rcParams.update({'font.size': font_size})
ax[0].plot(power[:,0])
ax[0].set_xlabel(r'Degree $\ell$')
ax[0].set_title('Velocity Power (total)')
ax[0].set_yscale('log')


ax[1].plot(power[:,1])
ax[1].set_xlabel(r'Degree $\ell$')
ax[1].set_title('Velocity Power (m=0)')
ax[1].set_yscale('log')

ax[2].plot(power[:,2])
ax[2].set_xlabel(r'Degree $\ell$')
ax[2].set_title('Velocity Power ( total - {m=0} )')
ax[2].set_yscale('log')

plt.tight_layout()
plt.show()

If desired, we can view the full $\ell-m$ spectrum as well, without integrating over $m$.  Have a look at both the Boussinesq and anelastic spectra and compare against their Shell_Slice outputs that you plotted earlier.

In [ ]:
fig, ax = plt.subplots()
mmax = ss.mmax
lmax = ss.lmax
power_spectrum = numpy.zeros((lmax+1,mmax+1),dtype='float64')

for i in range(1,4):   # i takes on values 1,2,3
    qind=ss.lut[i]
    complex_spectrum = ss.vals[:,:,rind,qind,tind]
    power_spectrum = power_spectrum+numpy.real(complex_spectrum)**2 + numpy.imag(complex_spectrum)**2

power_spectrum = numpy.transpose(power_spectrum)

tiny = 1e-6
img=ax.imshow(numpy.log10(power_spectrum+tiny), origin='lower')
ax.set_ylabel('Order m')
ax.set_xlabel(r'Degree $\ell$')
ax.set_title('Velocity Power Spectrum')

#colorbar ...
cbar = plt.colorbar(img) # ,shrink=0.5, aspect = 15)
cbar.set_label('Log Power')
        
tick_locator = ticker.MaxNLocator(nbins=5)
cbar.locator = tick_locator
cbar.update_ticks()
cbar.ax.tick_params()   #font size for the ticks


plt.show()